In [ ]:
!pip install --upgrade pip setuptools wheel
!pip install torch torchvision
!pip install pickle
!pip install matplotlib
!pip install cv2
!pip install numpy
!pip install ninja imageio imageio-ffmpeg==0.4.3 scipy==1.10.1

In [ ]:
!git clone https://github.com/NVlabs/stylegan3.git

import sys
sys.path.append('/content/stylegan3')
import pickle
import torch
import numpy as np
import matplotlib.pyplot as plt
from stylegan3 import legacy
import dnnlib

network_pkl = "/content/drive/MyDrive/ColabNotebooks/stylegan3-t-ffhqu-256x256.pkl"

with open(network_pkl, "rb") as f:
    data = pickle.load(f)


fatal: destination path 'stylegan3' already exists and is not an empty directory.


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [ ]:
gan = data["G_ema"].to("cuda")
gan.eval()

for param in gan.parameters():
    param.requires_grad = False


In [ ]:
import cv2

# generate two random faces (256x256)
def generate_and_save(seed, filename):
    torch.manual_seed(seed)
    z = torch.randn((1, 512), device='cuda')
    label = torch.zeros([1, gan.c_dim], device='cuda')

    with torch.no_grad():
        img = gan(z, label, truncation_psi=1.0, noise_mode='const')

    img = img.squeeze().cpu().numpy()
    img = np.transpose(img, (1, 2, 0))
    img[img > 1] = 1
    img[img < -1] = -1
    img = 255 * (img + 1) / 2
    img = img.astype(np.uint8)

    cv2.imwrite(filename, img[:, :, [2, 1, 0]])

# Generate faces
generate_and_save(seed=0, filename='face1.png')
generate_and_save(seed=1, filename='face2.png')

img1 = cv2.cvtColor(cv2.imread('face1.png'), cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(cv2.imread('face2.png'), cv2.COLOR_BGR2RGB)

fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(img1)
axs[0].set_title("Face 1 (seed=0)")
axs[0].axis("off")

axs[1].imshow(img2)
axs[1].set_title("Face 2 (seed=1)")
axs[1].axis("off")
plt.tight_layout()
plt.show()

Setting up PyTorch plugin "bias_act_plugin"... 

/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


Done.
Setting up PyTorch plugin "filtered_lrelu_plugin"... 

/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


In [ ]:

import pickle
import torch
import numpy as np
import cv2
import os
from tqdm import tqdm

# Animation between faces
torch.manual_seed(0)
z1 = torch.randn((1, gan.z_dim), device='cuda')
torch.manual_seed(1)
z2 = torch.randn((1, gan.z_dim), device='cuda')
label = torch.zeros([1, gan.c_dim], device='cuda')

os.makedirs("animatedFrames", exist_ok=True)
n_frames = 60

for i, alpha in enumerate(tqdm(np.linspace(0, 1, n_frames))):
    z = (1 - alpha) * z1 + alpha * z2
    with torch.no_grad():
        img = gan(z, label, truncation_psi=1.0, noise_mode='const')

    img = img.squeeze().cpu().numpy()
    img = np.transpose(img, (1, 2, 0))
    img = np.clip(img, -1, 1)
    img = ((img + 1) * 127.5).astype(np.uint8)

    filename = f"animatedFrames/frame_{i:03d}.png"
    cv2.imwrite(filename, img[:, :, [2, 1, 0]])

video_path = 'animatedVideo_256x256.mp4'
out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'mp4v'), 30, (256, 256))

for i in range(n_frames):
    frame = cv2.imread(f'animatedFrames/frame_{i:03d}.png')
    out.write(frame)

out.release()
print(f"Video saved: {video_path}")

In [ ]:
import os
from argparse import Namespace

import numpy as np
import torch
import sys
import dlib

sys.path.append(".")
sys.path.append("..")
sys.path.append('/content/drive/MyDrive/ColabNotebooks/PSPNet_simplified/PSPNet')

In [ ]:
from options.test_options import TestOptions
from models.psp import pSp
import cv2
from scripts.align_all_parallel import align_face
import torchvision.transforms as transforms

/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


In [ ]:


# Animated video from pictures of actors
test_opts = Namespace(checkpoint_path=None, couple_outputs=False, data_path='gt_images', exp_dir=None, latent_mask=None, mix_alpha=None, n_images=None, n_outputs_to_generate=5, resize_factors=None, resize_outputs=False, test_batch_size=2, test_workers=2)
test_opts.checkpoint_path = '/content/drive/MyDrive/ColabNotebooks/PSPNet_simplified/PSPNet/pretrained_models/psp_ffhq_encode.pt'
ckpt = torch.load(test_opts.checkpoint_path, map_location='cpu')
opts = ckpt['opts']
opts.update(vars(test_opts))
opts['output_size'] = 1024

opts = Namespace(**opts)


net = pSp(opts)
net.eval()
net.cuda()
#PSPNet + StyleGAN2 network

transform = transforms.Compose([
	transforms.Resize(size=(256, 256)),
	transforms.ToTensor(),
	transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])]
)


predictor = dlib.shape_predictor("/content/drive/MyDrive/ColabNotebooks/PSPNet_simplified/PSPNet/shape_predictor_68_face_landmarks.dat")


def get_latent_from_image(image_path, predictor, transform, net):
    aligned_image = align_face(filepath=image_path, predictor=predictor)
    aligned_image = aligned_image.convert("RGB")
    from_im = transform(aligned_image).unsqueeze(0)

    with torch.no_grad():
        input_tensor = from_im.float().cuda()
        _, latent_vector = net(input_tensor, randomize_noise=False, resize=False, return_latents=True)
    return latent_vector

img1_path = "/content/drive/MyDrive/ColabNotebooks/angelinajpeg.jpeg"
img2_path = "/content/drive/MyDrive/ColabNotebooks/brad.jpg"

latent1 = get_latent_from_image(img1_path, predictor, transform, net)
latent2 = get_latent_from_image(img2_path, predictor, transform, net)





Loading pSp from checkpoint: /content/drive/MyDrive/ColabNotebooks/PSPNet_simplified/PSPNet/pretrained_models/psp_ffhq_encode.pt


In [ ]:
# PSP net creates latent vectors with 18 layers, and stylegan3 expects 16 layers, so I had to go with stylegan2 which PSP was trained on.
!wget --content-disposition 'https://api.ngc.nvidia.com/v2/models/org/nvidia/team/research/stylegan2/1/files?redirect=true&path=stylegan2-ffhq-1024x1024.pkl' -o 'stylegan2-ffhq-1024x1024.pkl'

In [ ]:

!git clone https://github.com/NVlabs/stylegan2.git
import sys
sys.path.append('/content/stylegan2')

import dnnlib
import legacy
import pickle
import torch


with open("/content/stylegan2-ffhq-1024x1024.pkl", "rb") as f:
    G = pickle.load(f)['G_ema'].cuda()
G.eval()


fatal: destination path 'stylegan2' already exists and is not an empty directory.


Generator(
  (synthesis): SynthesisNetwork(
    w_dim=512, num_ws=18, img_resolution=1024, img_channels=3, num_fp16_res=4
    (b4): SynthesisBlock(
      resolution=4, architecture=skip
      (conv1): SynthesisLayer(
        in_channels=512, out_channels=512, w_dim=512, resolution=4, up=1, activation=lrelu
        (affine): FullyConnectedLayer(in_features=512, out_features=512, activation=linear)
      )
      (torgb): ToRGBLayer(
        in_channels=512, out_channels=3, w_dim=512
        (affine): FullyConnectedLayer(in_features=512, out_features=512, activation=linear)
      )
    )
    (b8): SynthesisBlock(
      resolution=8, architecture=skip
      (conv0): SynthesisLayer(
        in_channels=512, out_channels=512, w_dim=512, resolution=8, up=2, activation=lrelu
        (affine): FullyConnectedLayer(in_features=512, out_features=512, activation=linear)
      )
      (conv1): SynthesisLayer(
        in_channels=512, out_channels=512, w_dim=512, resolution=8, up=1, activation=lrelu


In [ ]:
print("latent1:", latent1.shape, latent1.device)
print("latent2:", latent2.shape, latent2.device)
print("First 1-2 values:", latent1[0, 0, :2])


latent1: torch.Size([1, 18, 512]) cuda:0
latent2: torch.Size([1, 18, 512]) cuda:0
First 1-2 values: tensor([-0.6538,  3.2965], device='cuda:0')


In [ ]:
import os
import numpy as np
import cv2
from tqdm import tqdm

os.makedirs("animatedActorsFrames", exist_ok=True)
n_frames = 60
label = torch.zeros([1, G.c_dim], device='cuda')

for i, alpha in enumerate(tqdm(np.linspace(0, 1, n_frames))):
    z = (1 - alpha) * latent1 + alpha * latent2
    with torch.no_grad():

      img = G.synthesis(z, noise_mode='const', input_is_latent=True)

    img = img.squeeze().cpu().numpy()
    img = np.transpose(img, (1, 2, 0))
    img = np.clip(img, -1, 1)
    img = ((img + 1) * 127.5).astype(np.uint8)

    filename = f"animatedActorsFrames/frame_{i:03d}.png"
    cv2.imwrite(filename, img[:, :, [2, 1, 0]])

video_path = 'animatedActorsVideo.mp4'
out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'mp4v'), 30, (1024, 1024))

for i in range(n_frames):
    frame = cv2.imread(f'animatedActorsFrames/frame_{i:03d}.png')
    out.write(frame)

out.release()
print(f"Video saved: {video_path}")

100%|██████████| 60/60 [00:07<00:00,  7.70it/s]


Video saved: animatedActorsVideo.mp4
